In [1]:
import matplotlib.pyplot as plt
from dipy.io.image import save_nifti
import numpy as np
import learning
from dipy.data import read_stanford_hardi, read_stanford_pve_maps
import dipy.reconst.dti as dti
import torch
import torch.nn as nn
from syn_data import SynDiffData
from water_correction import dehydrate
from dipy.segment.mask import median_otsu

from scipy.ndimage.morphology import binary_erosion

In [2]:
img, gtab = read_stanford_hardi()
dwi = img.get_data()
(csf, gm, wm) = [map.get_data()==1.0 for map in read_stanford_pve_maps()]

v_mask = np.zeros(csf.shape)
(x,y,z) = np.divide(csf.shape,2).astype(np.int)
v_mask[x-15:x+15,y-20:y+20,z-5:z+5] = 1
csf = np.logical_and(csf, v_mask)

wm[:,:,:25] = 0
gm[:,:,:25] = 0

csf, gm, wm = [binary_erosion(map) for map in (csf, gm, wm)]

save_nifti('csf', csf.astype(np.int), np.eye(4))
save_nifti('gm', gm.astype(np.int), np.eye(4))
save_nifti('wm', wm.astype(np.int), np.eye(4))
save_nifti('dwi', dwi, np.eye(4))


maskdata, mask = median_otsu(dwi, vol_idx=range(10, 50), median_radius=3,
                             numpass=1, autocrop=True, dilate=2)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/leon/.dipy/stanford_hardi 
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/leon/.dipy/stanford_hardi 


In [3]:
synDataLoader = SynDiffData(gtab, dwi, csf_mask=csf, gm_mask=gm, wm_mask=wm)
syn_diff_data, water_fraction = synDataLoader.create_waterfraction_data(sample_size=25000) # this may take a while

net = learning.Net(synDataLoader.datashape, 1)

optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
num_epochs = 100
criterion = nn.MSELoss()

learning.train(syn_diff_data, water_fraction, net, criterion, num_epochs, optimizer)

/miniconda3/envs/py36/lib/python3.6/site-packages/dipy/reconst/dti.py:1915: RuntimeWarning: divide by zero encountered in log
  D[..., 6] = -np.log(b0)


In [26]:
water_fraction, water_corrected = dehydrate(maskdata, mask, synDataLoader, net)

In [14]:
plt.figure('water fraction')
plt.imshow(water_fraction[:,:,30].T, cmap='gray')
plt.show()

(71, 87, 62)